# Параллельные вычисления

Материалы:
* Макрушин С.В. Лекция 10: Параллельные вычисления
* https://docs.python.org/3/library/multiprocessing.html

## Задачи для совместного разбора

In [1]:
import multiprocessing as mp

1. Посчитайте, сколько раз встречается каждый из символов (заглавные и строчные символы не различаются) в файле `Dostoevskiy Fedor. Prestuplenie i nakazanie - BooksCafe.Net.txt` и в файле `Dostoevskiy Fedor. Igrok - BooksCafe.Net.txt`. 

In [2]:
%%file count_letters.py
from collections import Counter

def count_letters(file):
    with open(file) as fp:
        text = fp.read().lower()
    return Counter(file)

Overwriting count_letters.py


In [3]:
from count_letters import count_letters

In [4]:
%%time
count_letters('../data/Dostoevskiy Fedor. Igrok - BooksCafe.Net.txt')
count_letters('../data/Dostoevskiy Fedor. Prestuplenie i nakazanie - BooksCafe.Net.txt')

Wall time: 82.9 ms


Counter({'.': 5,
         '/': 2,
         'd': 2,
         'a': 6,
         't': 6,
         'D': 1,
         'o': 5,
         's': 4,
         'e': 8,
         'v': 1,
         'k': 3,
         'i': 4,
         'y': 1,
         ' ': 6,
         'F': 1,
         'r': 2,
         'P': 1,
         'u': 1,
         'p': 1,
         'l': 1,
         'n': 3,
         'z': 1,
         '-': 1,
         'B': 1,
         'C': 1,
         'f': 1,
         'N': 1,
         'x': 1})

2. Решить задачу 1, распараллелив вычисления с помощью модуля `multiprocessing`. Для обработки каждого файла создать свой собственный процесс. 

*pool.map()* - возвращает результаты в том же порядке

**!** (но впринципе распараллеливание не должно использоваться, если порядок важен)

In [5]:
%%time
if __name__ == '__main__':

    files = [r'../data/Dostoevskiy Fedor. Igrok - BooksCafe.Net.txt', 
    r'../data/Dostoevskiy Fedor. Prestuplenie i nakazanie - BooksCafe.Net.txt']

    with mp.Pool(processes=len(files)) as pool:
        counters = pool.map(count_letters, files)

Wall time: 202 ms


In [6]:
print(f'{type(counters)}\n{counters[0]}\n{counters[1]}')

<class 'list'>
Counter({'o': 6, '.': 5, 't': 5, 'e': 4, ' ': 4, 'a': 3, 's': 3, 'k': 3, '/': 2, 'd': 2, 'r': 2, 'D': 1, 'v': 1, 'i': 1, 'y': 1, 'F': 1, 'I': 1, 'g': 1, '-': 1, 'B': 1, 'C': 1, 'f': 1, 'N': 1, 'x': 1})
Counter({'e': 8, 'a': 6, 't': 6, ' ': 6, '.': 5, 'o': 5, 's': 4, 'i': 4, 'k': 3, 'n': 3, '/': 2, 'd': 2, 'r': 2, 'D': 1, 'v': 1, 'y': 1, 'F': 1, 'P': 1, 'u': 1, 'p': 1, 'l': 1, 'z': 1, '-': 1, 'B': 1, 'C': 1, 'f': 1, 'N': 1, 'x': 1})


## Лабораторная работа 10

In [4]:
import csv

1. Разбейте файл `recipes_full.csv` на несколько (например, 8) примерно одинаковых по объему файлов c названиями `id_tag_nsteps_*.csv`. Каждый файл содержит 3 столбца: `id`, `tag` и `n_steps`, разделенных символом `;`. Для разбора строк используйте `csv.reader`.

__Важно__: вы не можете загружать в память весь файл сразу. Посмотреть на первые несколько строк файла вы можете, написав код, который считывает эти строки.

Подсказка: примерное кол-во строк в файле - 2.3 млн.

Фрагмент одного из файлов, которые должны получиться в результате:
```
id;tag;n_steps
137739;60-minutes-or-less;11
137739;time-to-make;11
137739;course;11
```


In [23]:
with open('../data/recipes_full.csv', encoding='utf-8') as f:
    file_reader = csv.reader(f, delimiter = ',')
    headers = next(file_reader)
    print(headers)
    for i, row in enumerate(file_reader, start=1):
        if i < 5:
            print(row)
        else:
            break

['name', 'id', 'minutes', 'contributor_id', 'submitted', 'tags', 'n_steps', 'steps', 'description', 'ingredients', 'n_ingredients']
['vant ivoire mickies nothing', '683970', '33', '803776', '2019-08-22', "['mexican', 'healthy-2', 'orange-roughy', 'chicken-thighs-legs', 'freezer', 'whitefish', 'pork-sausage']", '4', "['remove the fat cap and any large areas or pockets of external fat that can be easily trimmed away', 'cut the eggplants in half lengthwise and , taking care not to break the skins , gently scoop out the flesh', 'bake uncovered in a 350f for one hour or until golden and hot through', 'refrigerate']", "pat and gina neely and their family own and operate some of tennessee's best - and devilishly delicious - barbecue restaurants. but when they are relaxing at home in memphis, they love to create tasty dishes their whole family can enjoy. now they're ready to share secrets from their famous restaurant dishes.  here is the recipe for their barbecue chicken.  courtesy of food net

In [2]:
rows_number = 322356
columns = ['id', 'tag', 'n_steps']

new_file = open('id_tag_nsteps_1.csv', mode='w', encoding='utf-8')
file_writer = csv.writer(new_file, delimiter = ';', quotechar='\n')
file_writer.writerow(columns)

with open('../data/recipes_full.csv', encoding='utf-8') as f:
    file_reader = csv.reader(f, delimiter = ',')
    headers = next(file_reader)
    headers = [headers.index(column) for column in ['id', 'tags', 'n_steps']]

    for i, row in enumerate(file_reader, start=1):
        row_slice = [row[column] for column in range(len(row)) if column in headers]
        print(row_slice)
        for tag in row_slice[1][2:len(row_slice)-2].split("', '"):
            file_writer.writerow([row_slice[0], tag, row_slice[2]])

        if (i % rows_number == 0):
            print(i//rows_number)
            print(i)
            new_file.close()
            new_file = open(f'id_tag_nsteps_{i//rows_number+1}.csv', mode='w', encoding='utf-8')
            file_writer = csv.writer(new_file, delimiter = ';', quotechar='\n')
            file_writer.writerow(columns)

new_file.close()

1
322356
2
644712
3
967068
4
1289424
5
1611780
6
1934136


2. Напишите функцию, которая принимает на вход название файла, созданного в результате решения задачи 1, считает среднее значение количества шагов для каждого тэга и возвращает результат в виде словаря.

In [16]:
from typing import Dict

def tag_counter(file_name: str) -> Dict[str, float]:
    mean_value = {}
    
    with open(file_name) as file:
        file_reader = csv.reader(file, delimiter = ';')
        next(file_reader)
        for row in file_reader:
            r_id, tag, n_steps = row
            
            if tag not in mean_value.keys():
                mean_value[tag]= {'sum' : 0, 'count': 0}
            mean_value[tag]['sum'] += int(n_steps)
            mean_value[tag]['count'] += 1

    for tag, value in mean_value.items():
        mean_value[tag] = value['sum']/value['count']

    return mean_value

In [14]:
tag_counter('id_tag_nsteps_1.csv')

{'mexican': 4.0,
 'healthy-2': 4.0,
 'orange-roughy': 4.0,
 'chicken-thighs-legs': 4.0,
 'freezer': 4.0,
 'whitefish': 4.0,
 'pork-sausage': 4.0,
 'brunch': 1.0,
 'ham-and-bean-soup': 1.0,
 'colombian': 1.0,
 'savory-pies': 1.0,
 'refrigerator': 1.0,
 'australian': 1.0,
 'served-cold': 1.0,
 'passover': 1.0,
 'quick-breads': 1.0,
 'californian': 1.0,
 'namibian': 1.0,
 'candy': 1.0,
 "independence-day']": 1.0,
 'baking': 3.0,
 "pennsylvania-dutch']": 3.0,
 'weeknight': 8.0,
 '60-minutes-or-less': 8.0,
 'time-to-make': 8.0,
 'course': 8.0,
 'cuisine': 8.0,
 'preparation': 8.0,
 'occasion': 8.0,
 'n': 8.0}

3. Напишите функцию, которая считает среднее значение количества шагов для каждого тэга по всем файлам, полученным в задаче 1, и возвращает результат в виде словаря. Не используйте параллельных вычислений. При реализации выделите функцию, которая объединяет результаты обработки отдельных файлов. Модифицируйте код из задачи 2 таким образом, чтобы получить результат, имея результаты обработки отдельных файлов. Определите, за какое время задача решается для всех файлов.


In [ ]:
%%file tag_counter_all.py
import csv
from typing import Dict

def tag_counter_all(file_name: str) -> Dict[str, Dict[str, int]]:
    mean_value_dict = {}
    
    with open(file_name) as file:
        file_reader = csv.reader(file, delimiter = ';')
        next(file_reader)
        for row in file_reader:
            r_id, tag, n_steps = row
            
            if tag not in mean_value_dict.keys():
                mean_value_dict[tag]= {'sum' : 0, 'count': 0}
            mean_value_dict[tag]['sum'] += int(n_steps)
            mean_value_dict[tag]['count'] += 1

    return mean_value_dict

In [ ]:
from tag_counter_all import tag_counter_all
from typing import List

def mean_counter_all(files_list: List[str]) -> Dict[str, float]:
    files_list_dict = [tag_counter_all(item) for item in files_list]
    new_dict = {}
    for tag, value in files_list_dict.items():
        if tag not in new_dict:
            new_dict[tag] = value
        else:
            new_dict[tag]['count'] += value['count']
            new_dict[tag]['sum'] += value['sum']

    for tag, value in new_dict.items():
        new_dict[tag] = value['sum']/value['count']
    return new_dict

In [19]:
files_list = [f'id_tag_nsteps_{n}.csv' for n in range(1, 9)]
files_list

['id_tag_nsteps_1.csv',
 'id_tag_nsteps_2.csv',
 'id_tag_nsteps_3.csv',
 'id_tag_nsteps_4.csv',
 'id_tag_nsteps_5.csv',
 'id_tag_nsteps_6.csv',
 'id_tag_nsteps_7.csv',
 'id_tag_nsteps_8.csv']

In [ ]:
%%time
mean_counter_all(files_list)

4. Решите задачу 3, распараллелив вычисления с помощью модуля `multiprocessing`. Для обработки каждого файла создайте свой собственный процесс. Определите, за какое время задача решается для всех файлов.

In [ ]:
def mean_counter_all_new(files_list_dict) -> Dict[str, float]:
    new_dict = {}
    for tag, value in files_list_dict.items():
        if tag not in new_dict:
            new_dict[tag] = value
        else:
            new_dict[tag]['count'] += value['count']
            new_dict[tag]['sum'] += value['sum']

    for tag, value in new_dict.items():
        new_dict[tag] = value['sum']/value['count']
    return new_dict

In [ ]:
%%time
if __name__ == '__main__':
    with mp.Pool(processes=len(files_list)) as pool:
        counters = pool.map(tag_counter_all, files_list)

    mean_counter_all_new(counters)

5. Решите задачу 3, распараллелив вычисления с помощью модуля `multiprocessing`. Создайте фиксированное количество процессов (равное половине количества ядер на компьютере). При помощи очереди `multiprocessing.queue` передайте названия файлов для обработки процессам и при помощи другой очереди заберите от них ответы. 

In [ ]:
%%time
if __name__ == '__main__':
    processes = 4
    processes_list = []
    tasks_queue = mp.Queue()
    results_queue = mp.Queue()
    
    for file in files_list:
        tasks_queue.put(file)
    
    for i in range(processes):
        p = MyWorker(task_q,result_q)
        p.start()
        processes_list.append(p)
    with mp.Pool(processes=processes) as pool:
        results_queue.put(pool.map(tag_counter_all, tasks_queue.get()))

    mean_counter_all_new([results_queue.get() for _ in range(processes*2)])